<a href="https://colab.research.google.com/github/Jayveersinh-Raj/LoRA_implementation/blob/main/LoRA_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inference

In [39]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [40]:
!pip install datasets peft transformers rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=5361366d8a100d8019613a09e534556618c539114a86622e61d8b74dd3bc8b1c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
from datasets import load_dataset
dataset = load_dataset("jfleg")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/755 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/748 [00:00<?, ? examples/s]

In [8]:
test_data = dataset["test"]
test_data

Dataset({
    features: ['sentence', 'corrections'],
    num_rows: 748
})

In [9]:
test_data[0]

{'sentence': 'New and new technology has been introduced to the society .',
 'corrections': ['New technology has been introduced to society .',
  'New technology has been introduced into the society .',
  'Newer and newer technology has been introduced into society .',
  'Newer and newer technology has been introduced to the society .']}

### Load Model

In [11]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "Jayveersinh-Raj/bloom-sentence-correction"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=False, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
qa_model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/500 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/782 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.01G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/9.84M [00:00<?, ?B/s]

In [27]:
from IPython.display import display, Markdown

def make_inference(question):
  batch = tokenizer(f"### INCORRECT\n{question}\n\n### CORRECT\n", return_tensors='pt').to("cuda")

  with torch.cuda.amp.autocast():
    output_tokens = qa_model.generate(**batch, max_new_tokens=200)

  return tokenizer.decode(output_tokens[0], skip_special_tokens=True).split("\n")[-1]

In [32]:
input = []
preds = []
for text in test_data["sentence"][:100]:
  corrected = make_inference(text)
  preds.append(corrected)
  input.append(text)

In [54]:
gt = [sublist[0] for sublist in test_data["corrections"][:100]]
gt[0]

'New technology has been introduced to society .'

### CSV for evaluation

In [55]:
import pandas as pd
df = pd.DataFrame()
df["input"]=input
df["generated"]=preds
df["ground_truth"]=gt
df

,input,generated,ground_truth
0,New and new technology has been introduced to ...,New and new technology has been introduced to ...,New technology has been introduced to society .
1,One possible outcome is that an environmentall...,One possible outcome is that an environmentall...,One possible outcome is that an environmentall...
2,"Every person needs to know a bit about math , ...","Every person needs to know a bit about math , ...","Every person needs to know a bit about math , ..."
3,While the travel company will most likely show...,While the travel company will most likely show...,While the travel company will most likely show...
4,Disadvantage is parking their car is very diff...,Disadvantage is parking their car is very diff...,A disadvantage is that parking their cars is v...
...,...,...,...
95,Reasons is pay for them and teach the more tha...,Reasons is pay for them and teach the more tha...,The reasons for this are that their pay is hig...
96,I totally agree that the youth today are not s...,I totally agree that the youth today are not s...,I totally agree that the youth today are not s...
97,"Nowadays , the society is changing drasticly .","Nowadays , the society is changing drasticly .","Nowadays , society is changing drastically ."
98,"and through text book , students are more enga...","and through text book , students are more enga...","And through the text book , students are more ..."


In [56]:
df.to_csv("predictions.csv", index=False)

### Rouge scores

In [57]:
from rouge_score import rouge_scorer
import pandas as pd

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []

# Iterate through rows and calculate ROUGE scores
for index, row in df.iterrows():
    reference = row['ground_truth']  # Ground truth reference
    prediction = row['generated']  # Model-generated prediction

    # Calculate ROUGE scores
    scores = scorer.score(reference, prediction)

    rouge_scores.append(scores)

# Create a new DataFrame to store ROUGE scores
rouge_df = pd.DataFrame(rouge_scores)

In [58]:
rouge1 = list(rouge_df["rouge1"])
rouge2 = list(rouge_df["rouge2"])
rougeL = list(rouge_df["rougeL"])

def mean_rouge(ls : list):
    sum = 0
    count = 0
    for scores in ls:
        count+=1
        sum+=scores[-1]
    return sum/count
avg_rouge1 = mean_rouge(rouge1)
avg_rouge2 = mean_rouge(rouge2)
avg_rougeL = mean_rouge(rougeL)

avg_rouge1, avg_rouge2, avg_rougeL

(0.7916041032531175, 0.6870515830569782, 0.7819656758047572)

### BERT similarity

In [59]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import warnings

# Initialize the BERT model and tokenizer outside the loop
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.to("cuda")

def bert_text_similarity(reference, candidate, tokenizer, model):
    tokens_ref = tokenizer.encode(reference, add_special_tokens=True, return_tensors="pt").to("cuda")
    tokens_cand = tokenizer.encode(candidate, add_special_tokens=True, return_tensors="pt").to("cuda")

    with torch.no_grad():
        embeddings_ref = model(tokens_ref)[0][:, 0, :]
        embeddings_cand = model(tokens_cand)[0][:, 0, :]

    similarity_score = cosine_similarity(embeddings_ref.cpu(), embeddings_cand.cpu())[0][0]

    return similarity_score

def calculate_mean_similarity(df, tokenizer, model):
    total_similarity = 0.0
    num_pairs = len(df)

    for index, row in df.iterrows():
        human_response = row["ground_truth"]
        model_response = row["generated"]

        similarity_score = bert_text_similarity(human_response, model_response, tokenizer, model)
        total_similarity += similarity_score

    mean_similarity = total_similarity / num_pairs
    return mean_similarity

# Calculate and print the mean similarity
mean_similarity = calculate_mean_similarity(df, tokenizer, model)
print(f"Mean BERT Text Similarity Score: {mean_similarity:.2f}")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Mean BERT Text Similarity Score: 0.88
